DATA AUGMENTATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

We will try to use the give HR images in the the training set to augment more images and then downgrade them to Bicubical to create corresponding LR images.

In [ ]:
import os
import random
from PIL import Image, ImageFilter
import torchvision.transforms as T
import torchvision.transforms.functional as F
from tqdm import tqdm
%cd /content/drive/MyDrive/Colab Notebooks

HR_DIR = 'scaling_4x/train/HR'
AUG_HR_DIR = 'data/HR'
AUG_LR_DIR = 'data/LR'
os.makedirs(AUG_HR_DIR, exist_ok=True)
os.makedirs(AUG_LR_DIR, exist_ok=True)


def augment_hr_image(hr_img):

    hr_crop = T.RandomCrop(256)(hr_img)

    #  flip
    if random.random() > 0.5:
        hr_crop = F.hflip(hr_crop)

    # rotation
    if random.random() > 0.7:
        hr_crop = F.rotate(hr_crop, angle=random.choice([90, 180, 270]))

    # jitter
    color_aug = T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
    hr_crop = color_aug(hr_crop)

    return hr_crop

def degrade_hr_to_lr(hr_img):

    lr_img = hr_img.resize((64, 64), Image.BICUBIC)


    if random.random() > 0.7:
        lr_img = lr_img.filter(ImageFilter.GaussianBlur(radius=random.uniform(0.2, 0.5)))

    return lr_img

n_aug_per_image = 11
image_files = [f for f in os.listdir(HR_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

for img_file in tqdm(image_files):
    hr_path = os.path.join(HR_DIR, img_file)
    hr_img = Image.open(hr_path).convert('RGB')

    base_name = os.path.splitext(img_file)[0]

    for i in range(n_aug_per_image):
        aug_hr = augment_hr_image(hr_img)
        aug_lr = degrade_hr_to_lr(aug_hr)

        hr_save_path = os.path.join(AUG_HR_DIR, f"{base_name}_aug{i}.png")
        lr_save_path = os.path.join(AUG_LR_DIR, f"{base_name}_aug{i}.png")

        aug_hr.save(hr_save_path)
        aug_lr.save(lr_save_path)


/content/drive/MyDrive/Colab Notebooks


100%|██████████| 80/80 [01:49<00:00,  1.37s/it]


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks
import os
import shutil
import random
from collections import defaultdict

HR_DIR = 'data/HR'
LR_DIR = 'data/LR'
VAL_HR_DIR = 'dataset/val/HR'
VAL_LR_DIR = 'dataset/val/LR'
TRAIN_HR_DIR = 'dataset/train/HR'
TRAIN_LR_DIR = 'dataset/train/LR'


os.makedirs(VAL_HR_DIR, exist_ok=True)
os.makedirs(VAL_LR_DIR, exist_ok=True)
os.makedirs(TRAIN_HR_DIR, exist_ok=True)
os.makedirs(TRAIN_LR_DIR, exist_ok=True)

grouped = defaultdict(list)
for fname in os.listdir(HR_DIR):
    if fname.endswith(".png"):
        base = fname.split('_aug')[0]
        grouped[base].append(fname)

validation_set = []
remaining_set = []

for group_imgs in grouped.values():
    if len(group_imgs) == 0:
        continue
    val_img = random.choice(group_imgs)
    validation_set.append(val_img)
    remaining_imgs = [img for img in group_imgs if img != val_img]
    remaining_set.extend(remaining_imgs)


needed = 100 - len(validation_set)
if needed > 0:
    extra_val_imgs = random.sample(remaining_set, needed)
    validation_set.extend(extra_val_imgs)
    remaining_set = [img for img in remaining_set if img not in extra_val_imgs]


def move_set(file_list, hr_src, lr_src, hr_dst, lr_dst):
    for fname in file_list:
        shutil.copy(os.path.join(hr_src, fname), os.path.join(hr_dst, fname))
        shutil.copy(os.path.join(lr_src, fname), os.path.join(lr_dst, fname))

move_set(validation_set, HR_DIR, LR_DIR, VAL_HR_DIR, VAL_LR_DIR)
move_set(remaining_set, HR_DIR, LR_DIR, TRAIN_HR_DIR, TRAIN_LR_DIR)

print(f"Validation set: {len(validation_set)} images")
print(f"Training set: {len(remaining_set)} images")


/content/drive/MyDrive/Colab Notebooks
Validation set: 100 images
Training set: 780 images


In [ ]:
HR_DIR = '/dataset/train/HR'
LR_DIR = '/dataset/train/LR'


In [ ]:
import os
import random

HR_DIR = '/content/drive/MyDrive/Colab Notebooks/dataset/train/HR'
LR_DIR = '/content/drive/MyDrive/Colab Notebooks/dataset/train/LR'

%cd /content/drive/MyDrive/Colab Notebooks

file_list = [f for f in os.listdir(HR_DIR) if f.endswith('.png')]
random.shuffle(file_list)

for fname in file_list[:5]:
    hr_path = os.path.join(HR_DIR, fname)
    lr_path = os.path.join(LR_DIR, fname)
    print(f"HR: {hr_path}\nLR: {lr_path}\n")



/content/drive/MyDrive/Colab Notebooks
HR: /content/drive/MyDrive/Colab Notebooks/dataset/train/HR/227092_aug8.png
LR: /content/drive/MyDrive/Colab Notebooks/dataset/train/LR/227092_aug8.png

HR: /content/drive/MyDrive/Colab Notebooks/dataset/train/HR/119082_aug4.png
LR: /content/drive/MyDrive/Colab Notebooks/dataset/train/LR/119082_aug4.png

HR: /content/drive/MyDrive/Colab Notebooks/dataset/train/HR/299086_aug9.png
LR: /content/drive/MyDrive/Colab Notebooks/dataset/train/LR/299086_aug9.png

HR: /content/drive/MyDrive/Colab Notebooks/dataset/train/HR/123074_aug7.png
LR: /content/drive/MyDrive/Colab Notebooks/dataset/train/LR/123074_aug7.png

HR: /content/drive/MyDrive/Colab Notebooks/dataset/train/HR/182053_aug6.png
LR: /content/drive/MyDrive/Colab Notebooks/dataset/train/LR/182053_aug6.png

